In [2]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import glob
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations

In [ ]:
def build_generator():
    noise_shape = (100,)
    model = tf.keras.Sequential()

    model.add(layers.Dense(6 * 6 * 512, input_shape = noise_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((6, 6, 512)))
    
    model.add(layers.Conv2DTranspose(256, (5, 5), strides = (2, 2), padding = 'same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides = (2, 2), padding = 'same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64, (5, 5), strides = (2, 2), padding = 'same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides = (2, 2), padding = 'same', activation='tanh'))

    return model

def build_discriminator():
        
    img_shape = (96, 96, 3)
    model = tf.keras.Sequential()

    model.add(layers.Conv2D(64, (5, 5), strides = (2, 2), input_shape=img_shape, padding = 'same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides = (2, 2), input_shape=img_shape, padding = 'same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides = (2, 2), input_shape=img_shape, padding = 'same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, (5, 5), strides = (2, 2), input_shape=img_shape, padding = 'same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

def get_data(imgPath,dirNum):
    basePath = '/content/drive/My Drive/Colab Notebooks/DCGAN-data/'
    basePath = basePath + str(dirNum) + '/'
    X = []
    for path in tqdm(imgPath):
        img = cv2.imread(basePath + path)
        img = np.asarray(img)
        img = cv2.resize(img,(96,96))
        X.append(img)
    return X

def discriminator_loss(real_output, fake_output):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def gene_imgs(count,g):
    noise = np.random.normal(0, 1, (count, 100))
    return g.predict(noise)
    
def save_imgs(epoch,g):
    r, c = 5, 5

    imgs = gene_imgs(r*c,g)
    imgs = 127.5 * imgs + 127.5

    nindex, height, width, intensity = imgs.shape
    nrows = nindex // c
    assert nindex == nrows * c
        
    gallery = (imgs.reshape(nrows, c, height, width, intensity)
                  .swapaxes(1, 2)
                  .reshape(height * nrows, width * c, intensity))
    path = '/content/drive/My Drive/Colab Notebooks/gantest' + '/gallery'
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(path + f"/{epoch}.jpg", gallery)

BATCH_SIZE = 64
generator = build_generator()
discriminator = build_discriminator()
generator_optimizer = tf.keras.optimizers.Adam(0.0002,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002,0.5)
checkpoint_dir = '/content/drive/My Drive/Colab Notebooks/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)
        save_imgs(epoch,generator)
        if (epoch + 1) % 25 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))


pathList = os.listdir('/content/drive/My Drive/Colab Notebooks/DCGAN-data/1/')
X_train1 = get_data(pathList,1)
pathList = os.listdir('/content/drive/My Drive/Colab Notebooks/DCGAN-data/2/')
X_train2 = get_data(pathList,2)
pathList = os.listdir('/content/drive/My Drive/Colab Notebooks/DCGAN-data/3/')
X_train3 = get_data(pathList,3)
pathList = os.listdir('/content/drive/My Drive/Colab Notebooks/DCGAN-data/4/')
X_train4 = get_data(pathList,4)
X_train = X_train1 + X_train2 + X_train3 + X_train4
X_train = np.asarray(X_train)
'''
np.save('/content/drive/My Drive/Colab Notebooks/data.npy',X_train)  #save the data 
print('data_save')

X_train = np.load('/content/drive/My Drive/Colab Notebooks/data.npy')   #preload the data 
print('data_load')
'''
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).batch(BATCH_SIZE)
train(train_dataset,50)